In [1]:
import numpy as np

# Case I : Basic RNN, no cost module, with numpy

X = np.array([[6, 5, 4],   # t=0
              [3, 2, 1],   # t=1
              [0, 1, 2],   # t=2
              [3, 4, 5]])  # t=3

n_steps, n_features = X.shape
n_neurons = 5

W = np.random.random((n_neurons, n_features))
U = np.random.random((n_neurons, n_neurons))
b = np.zeros(n_neurons)

state_t = np.random.random(n_neurons)

In [2]:
output = []

for x_t in X:
    output_t = np.tanh(np.dot(W, x_t) + np.dot(U, state_t) + b)
    output.append(output_t)
    state_t = output_t

print('n_steps={}, n_features={}, n_neurons={}'.format(n_steps, n_features, n_neurons))
for t, y in enumerate(output):
    print('time step={}, y={}'.format(t, y))

n_steps=4, n_features=3, n_neurons=5
time step=0, y=[0.99999991 0.99999998 0.99999999 1.         1.        ]
time step=1, y=[0.99996995 0.99997019 0.99993962 0.99998959 0.99999671]
time step=2, y=[0.99984901 0.99994492 0.99991286 0.99995508 0.99894231]
time step=3, y=[0.99999992 0.99999999 0.99999999 1.         1.        ]


In [3]:
import tensorflow as tf

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
# Case II : Basic RNN with tensorflow symbols

n_features = 3
n_neurons = 5
n_steps = 2

X0 = tf.placeholder(tf.float32, [None, n_features])
X1 = tf.placeholder(tf.float32, [None, n_features])

W = tf.Variable(tf.random_normal(shape=[n_features, n_neurons], dtype=tf.float32))
U = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([n_neurons], dtype=tf.float32))

Y0 = tf.tanh(X0 @ W + b)
Y1 = tf.tanh(X1 @ W + Y0 @ U + b)

init = tf.global_variables_initializer()

In [5]:
X0_batch = np.array([[0, 1, 2],
                     [3, 4, 5],
                     [6, 7, 8],
                     [9, 0, 1]])  # t=0
X1_batch = np.array([[9, 8, 7],
                     [0, 0, 0],
                     [6, 5, 4],
                     [3, 2, 1]])  # t=1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

print('time step={}, y=\n{}'.format(0, Y0_val))
print('time step={}, y=\n{}'.format(1, Y1_val))

time step=0, y=
[[-0.8674343  -0.17522326 -0.7387707  -0.92735523  0.3188996 ]
 [-0.99518144 -0.9994043  -0.899996   -1.          0.13633975]
 [-0.99983567 -0.9999998  -0.9637886  -1.         -0.05597491]
 [-0.9932283  -0.9895281  -0.99669033 -1.         -0.9999122 ]]
time step=1, y=
[[-0.99916446 -1.         -0.31463987 -1.         -0.9979193 ]
 [ 0.60434014 -0.9998892   0.8899035   0.19729765 -0.9966838 ]
 [-0.88657403 -1.          0.89533746 -1.         -0.9991767 ]
 [-0.15822153 -0.9999997   0.99105066 -0.99999684 -0.99802035]]


In [6]:
# Case III : Basic RNN with tf.nn.static_rnn

reset_graph()

n_features = 3
n_neurons = 5
n_steps = 2

X = tf.placeholder(tf.float32, [None, n_steps, n_features])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

basic_cell  = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.nn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

init = tf.global_variables_initializer()

In [7]:
X_batch = np.array(
     # t = 0,     t = 1
    [[[0, 1, 2], [9, 8, 7]], # sample 1
     [[3, 4, 5], [0, 0, 0]], # sample 2
     [[6, 7, 8], [6, 5, 4]], # sample 3
     [[9, 0, 1], [3, 2, 1]]] # sample 4
)

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

print(outputs_val)

[[[-0.45652324 -0.68064123  0.40938237  0.63104504 -0.45732826]
  [-0.9428799  -0.9998869   0.94055814  0.9999985  -0.9999997 ]]

 [[-0.8001535  -0.9921827   0.7817797   0.9971032  -0.9964609 ]
  [-0.637116    0.11300927  0.5798437   0.4310559  -0.6371699 ]]

 [[-0.93605185 -0.9998379   0.9308867   0.9999815  -0.99998295]
  [-0.9165386  -0.9945604   0.896054    0.99987197 -0.9999751 ]]

 [[ 0.9927369  -0.9981933  -0.55543643  0.9989031  -0.9953323 ]
  [-0.02746338 -0.73191994  0.7827872   0.9525682  -0.9781773 ]]]


In [9]:
# Case VI : Basic RNN with tf.nn.dynamic_rnn

reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_features])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

init = tf.global_variables_initializer()

In [11]:
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

print(outputs_val)

[[[-0.85115266  0.87358344  0.5802911   0.8954789  -0.0557505 ]
  [-0.999996    0.99999577  0.9981815   1.          0.37679607]]

 [[-0.9983293   0.9992038   0.98071456  0.999985    0.25192663]
  [-0.7081804  -0.0772338  -0.85227895  0.5845349  -0.78780943]]

 [[-0.9999827   0.99999535  0.9992863   1.          0.5159072 ]
  [-0.9993956   0.9984095   0.83422637  0.99999976 -0.47325212]]

 [[ 0.87888587  0.07356028  0.97216916  0.9998546  -0.7351168 ]
  [-0.9134514   0.3600957   0.7624866   0.99817705  0.80142   ]]]
